In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내고 계신가요? 도움이 필요하거나 궁금한 점이 있다면 언제든지 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 12, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CcY3Sbow3tnzhDA2YmRynmFohmSuj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--36b662bb-fdf3-4c4f-9de6-df4c70827775-0', usage_metadata={'input_tokens': 12, 'output_tokens': 35, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [4]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CcY3T7zfQOfxH5ZNtGFtSu31izIBL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4dacdaf0-dc51-4b7b-8f5b-4114f1389432-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_73y3iEVZH0iUXB6n30FWZtP1', 'type': 'tool_call'}],

In [6]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-11-16 23:30:08 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CcY3T7zfQOfxH5ZNtGFtSu31izIBL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4dacdaf0-dc51-4b7b-8f5b-4114f1389432-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_73y3iEVZH0iUXB6n30FWZtP1', 'type': 'tool_call'}

In [7]:
llm_with_tools.invoke(messages)

AIMessage(content='현재 부산의 시각은 2025년 11월 16일 23:30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 192, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CcY3U7aT18BJwt8JALKM9MLTwcVos', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--0b5dcef9-6f0c-44b9-9eec-6d2b1f110c48-0', usage_metadata={'input_tokens': 192, 'output_tokens': 23, 'total_tokens': 215, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [9]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [10]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 283, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CcY3VPh84zEQJnQ39CROlqTv53Ksp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--9ffe314f-adae-42c5-b640-f891a0bcdc0a-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_zFVY0PEd9YRqR1yVGKS7kSC1', 'type': 'tool_call'}] usage_metadata={'input_tokens': 283, 'output_tokens': 27, 'total_tokens': 310, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}

In [11]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-10-15 00:00:00-04:00 | 434.9  | 440.51 | 426.33 |  435.15 | 7.15582e+07 |           0 |              0 |\n| 2025-10-16 00:00:00-04:00 | 434.73 | 439.35 | 421.31 |  428.75 | 7.71899e+07 |           0 |              0 |\n| 2025-10-17 00:00:00-04:00 | 425.5  | 441.46 | 423.6  |  439.31 | 8.93316e+07 |           0 |              0 |\n| 2025-10-20 00:00:00-04:00 | 443.87 | 449.8  | 440.61 |  447.43 | 6.3719e+07  |           0 |              0 |\n| 2025-10-21 00:00:00-04:00 | 445.76 | 449.3  | 442.05 |  442.6  | 5.44122e+07 |           0 |              0 |\n| 2025-10-22 00:00:00-04:00 | 443.45 | 445.54 | 429    |  438.97 | 8.40235e+07 |           0 |              0 |\n| 2025-10-23 00:00:00-04:0

In [12]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전 테슬라(TSLA)의 주가는 약 435.15달러로 마감하였고, 현재 주가는 404.35달러입니다. 따라서 한 달 사이에 주가는 약 30.8달러 하락하였습니다. 즉, 최근 한 달 동안 테슬라의 주가는 내렸습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1705, 'total_tokens': 1778, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CcY3XvcKKwkzJTvLMlfTCutWKyZnh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--0d850a55-1f81-4440-8456-81476eabd176-0', usage_metadata={'input_tokens': 1705, 'output_tokens': 73, 'total_tokens': 1778, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점에 대해 이야기해줘.")]):
    print(c.content, end='|') 

|안|녕하세요|!| 한국| 사회|에는| 여러| 가지| 문제|점|이| 존재|합니다|.| 몇| 가지| 주요| 문제|를| 말씀|드|리|면|:

|1|.| **|고|용| 불|안|과| 청|년| 실|업|**|:| 많은| 청|년|들이| 안정|적인| 일|자|리를| 찾|기| 어렵|고|,| 이는| 경제|적| 걱|정을| 증|대|시|킵|니다|.| 특히| 대|기업| 위|주의| 채|용| 문화|가| 중|소|기업|이나| 창|업|을| 어려|운| 상황|으로| 만들|고| 있습니다|.

|2|.| **|주|거| 문제|**|:| 서울|과| 같은| 대|도|시|의| 주|거|비|가| 급|격|히| 상승|하면서| 젊|은| 세|대|가| 자신|만|의| 집|을| 마련|하기| 어려|운| 실|정|입니다|.| 전|세|와| 월|세|의| 부담|이| 큰| 문제|로| 대|두|되고| 있습니다|.

|3|.| **|사회|적| 양|극|화|**|:| 소|득| 격|차|가| 확대|되|면서| 부|유|층|과| 저|소|득|층| 간|의| 간|극|이| 심|화|되고| 있습니다|.| 이는| 교육|,| 의료|,| 주|거| 등| 다양한| 분야|에서| 불|평|등|을| 초|래|합니다|.

|4|.| **|정|신| 건강| 문제|**|:| 경쟁|이| 치|열|한| 사회|적| 환경| 속|에서| 스트|레스|와| 우|울|증| 등| 정신|적| 문제|를| 겪|는| 사람들이| 많|습니다|.| 그러나| 여|전히| 이에| 대한| 인|식|이| 부족|해| 충분|한| 지원|을| 받|기| 어렵|습니다|.

|5|.| **|여|성| 인|권| 문제|**|:| 성|별|에| 따른| 불|평|등|과| 성|폭|력| 문제|는| 여|전히| 심|각|합니다|.| 직|장에서|의| 성|차|별|,| 가|정| 내| 폭|력| 등| 여러| 문제가| 사회|적으로| 논|의|되고| 있지만|,| 해결|하기| 위한| 노|력이| 부족|한| 상황|입니다|.

|6|.| **|노|인| 문제|**|:| 고|령|화| 사회|로| 접|어|들|면서| 노|인| 복|지|와| 관련|된| 문제|도| 증가|하고| 있습니다|.| 홀|몸| 노|

In [14]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

response = llm_with_tools.stream(messages)

# 파편화된 tool_call 청크를 하나로 합치기 
is_first = True
for chunk in response:    
    print("chunk type: ", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'type': 'tool_c

In [15]:
gathered

AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'service_tier': 'default'}, id='lc_run--c368810c-ee52-48c9-a99a-e5e2d29c0890', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 203, 'output_tokens': 23, 'total_tokens': 226, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'index': 0, 'type': 'tool_call_chunk'}], chunk_position='last')

In [16]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 이름으로 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-11-16 23:36:03 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'service_tier': 'default'}, id='lc_run--c368810c-ee52-48c9-a99a-e5e2d29c0890', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 203, 'output_tokens': 23, 'total_tokens': 226, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_FbBCcAYLxfVjHYL1Qp9i94rf', 'index': 0, 'type': 'tool_call_chunk'}

In [17]:
for c in llm_with_tools.stream(messages):
    print(c.content, end='|')

|부|산|은| 현재| |202|5|년| |11|월| |16|일| |23|시| |36|분|입니다|.||||